In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import time
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation

## <b>1.  Importing Data to Python</font></b>
---

In [3]:
clean_application_train = pd.read_csv('D:/Code Skripsi - RL and Ensemble/data/raw/merged_dataset.csv')

In [4]:
clean_application_train.shape

(307511, 48)

In [5]:
clean_application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FIRST_MONTH_BALANCE,LAST_MONTH_BALANCE,NUM_PREV_LOANS,SUM_PREV_CREDIT,AVG_PREV_CREDIT,RECENT_LOAN_DAYS,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_PAYMENT,DAYS_ENTRY_PAYMENT
0,100002,1,Cash loans,N,Y,0,41947875.0,8.422667e+07,5116708.575,72709650.0,...,-47.0,0.0,1.0,3.709124e+07,3.709124e+07,-606.0,-19.0,-1.0,2.394498e+06,0.0
1,100003,0,Cash loans,N,N,0,55930500.0,2.679490e+08,7394944.275,233975925.0,...,NaN,NaN,3.0,3.009005e+08,1.003002e+08,-2341.0,-77.0,-18.0,1.627339e+07,0.0
2,100004,0,Revolving loans,Y,Y,0,13982625.0,2.796525e+07,1398262.500,27965250.0,...,NaN,NaN,1.0,4.164958e+06,4.164958e+06,-815.0,-27.0,-24.0,1.469969e+06,0.0
3,100006,0,Cash loans,N,Y,0,27965250.0,6.477218e+07,6149558.475,61523550.0,...,NaN,NaN,9.0,5.438225e+08,6.042472e+07,-617.0,-20.0,-1.0,5.011875e+07,0.0
4,100007,0,Cash loans,N,Y,0,25168725.0,1.062680e+08,4529438.325,106267950.0,...,NaN,NaN,6.0,2.071153e+08,3.451922e+07,-2357.0,-77.0,-1.0,2.417760e+06,0.0


## <b>2.  Check Missing Value Percentage</font></b>
---
#### Before handling missing values, first check the percentage of missing data:

In [6]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    missing_percentage = (missing_values / len(df)) * 100
    missing_data = pd.DataFrame({
        'Missing Values': missing_values,
        'Percentage': missing_percentage
    })

    missing_data = missing_data[missing_data['Missing Values'] > 0].sort_values(
        by='Percentage', ascending=False)

    if missing_data.empty:
        print("No missing values found in the DataFrame.")
    else:
        print("Missing Values Summary:")
        print(missing_data)

In [7]:
check_missing_values(clean_application_train)

Missing Values Summary:
                        Missing Values  Percentage
LAST_MONTH_BALANCE              215280   70.007252
FIRST_MONTH_BALANCE             215280   70.007252
OWN_CAR_AGE                     202929   65.990810
OCCUPATION_TYPE                  96391   31.345545
AVG_CREDIT                       44021   14.315260
SUM_CREDIT                       44020   14.314935
NUM_BUREAU_LOANS                 44020   14.314935
LAST_CREDIT_DAYS                 44020   14.314935
OVERDUE_MONTHS                   44020   14.314935
FIRST_CREDIT_DAYS                44020   14.314935
ACTIVE_LOANS                     44020   14.314935
MONTHS_BALANCE_max               20544    6.680737
MONTHS_BALANCE_min               20544    6.680737
AMT_PAYMENT                      18113    5.890196
RECENT_LOAN_DAYS                 16454    5.350703
NUM_PREV_LOANS                   16454    5.350703
SUM_PREV_CREDIT                  16454    5.350703
AVG_PREV_CREDIT                  16454    5.350703
DAYS_EN

## <b>3.  Divide Numerical & Categorical Features</font></b>
---
#### Before handling missing values, I need to separate numerical and categorical features since they require different imputation methods.




In [8]:
# Identify numerical and categorical features
numerical_features = clean_application_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = clean_application_train.select_dtypes(include=['object']).columns.tolist()

# Display feature lists
print(f"Numerical Features ({len(numerical_features)}):\n", numerical_features)
print(f"\nCategorical Features ({len(categorical_features)}):\n", categorical_features)

Numerical Features (37):
 ['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'DAYS_LAST_PHONE_CHANGE', 'NUM_BUREAU_LOANS', 'SUM_CREDIT', 'AVG_CREDIT', 'FIRST_CREDIT_DAYS', 'LAST_CREDIT_DAYS', 'ACTIVE_LOANS', 'OVERDUE_MONTHS', 'FIRST_MONTH_BALANCE', 'LAST_MONTH_BALANCE', 'NUM_PREV_LOANS', 'SUM_PREV_CREDIT', 'AVG_PREV_CREDIT', 'RECENT_LOAN_DAYS', 'MONTHS_BALANCE_min', 'MONTHS_BALANCE_max', 'AMT_PAYMENT', 'DAYS_ENTRY_PAYMENT']

Categorical Features (11):
 ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE']


Numerical Features:

* Low missing (<5%) → Fill with median.
* Moderate missing (5–30%) → Use group-based median or KNN.
* High missing (>30%) → Drop if unimportant or use advanced imputation.

Categorical Features:

* Low missing (<5%) → Fill with mode (most frequent value).
* Moderate missing (5–30%) → Fill using predictive imputation.
* High missing (>30%) → Drop unless crucial.

## Numerical Low missing (<5%)
---

In [9]:
# Calculate the percentage of missing values in numerical columns
missing_numerical = clean_application_train.select_dtypes(include=['number']).isnull().mean() * 100

In [10]:
# Low missing (<5%)

low_missing_numerical = missing_numerical[(missing_numerical < 5) & (missing_numerical > 0)]
print("Numerical Features with Low Missing Values (0% < Missing < 5%):\n", low_missing_numerical)

# fill numerical features with low missing values (less than 5%) using the median
for feature in low_missing_numerical.index:
    if clean_application_train[feature].isnull().any():  # Check if there are any missing values
        clean_application_train[feature] = clean_application_train[feature].fillna(clean_application_train[feature].median())

Numerical Features with Low Missing Values (0% < Missing < 5%):
 AMT_ANNUITY               0.003902
AMT_GOODS_PRICE           0.090403
CNT_FAM_MEMBERS           0.000650
DAYS_LAST_PHONE_CHANGE    0.000325
dtype: float64


In [11]:
# Impute Missing Values
impute_median_features = [
    'AMT_ANNUITY',
    'AMT_GOODS_PRICE',
    'CNT_FAM_MEMBERS',
    'DAYS_LAST_PHONE_CHANGE'
]


for feature in impute_median_features:
    imputed_col_name = feature + '_imputed'
    clean_application_train[imputed_col_name] = clean_application_train[feature].fillna(clean_application_train[feature].median()) # Example imputation

# Replace Original Columns
for feature in impute_median_features:
    imputed_col_name = feature + '_imputed'
    if imputed_col_name in clean_application_train.columns:
        clean_application_train[feature] = clean_application_train[imputed_col_name]
        clean_application_train.drop(columns=[imputed_col_name], inplace=True)
    else:
        print(f"Imputed column '{imputed_col_name}' not found. Skipping replacement for '{feature}'.")

In [12]:
check_missing_values(clean_application_train)

Missing Values Summary:
                     Missing Values  Percentage
LAST_MONTH_BALANCE           215280   70.007252
FIRST_MONTH_BALANCE          215280   70.007252
OWN_CAR_AGE                  202929   65.990810
OCCUPATION_TYPE               96391   31.345545
AVG_CREDIT                    44021   14.315260
NUM_BUREAU_LOANS              44020   14.314935
LAST_CREDIT_DAYS              44020   14.314935
SUM_CREDIT                    44020   14.314935
OVERDUE_MONTHS                44020   14.314935
FIRST_CREDIT_DAYS             44020   14.314935
ACTIVE_LOANS                  44020   14.314935
MONTHS_BALANCE_max            20544    6.680737
MONTHS_BALANCE_min            20544    6.680737
AMT_PAYMENT                   18113    5.890196
RECENT_LOAN_DAYS              16454    5.350703
NUM_PREV_LOANS                16454    5.350703
SUM_PREV_CREDIT               16454    5.350703
AVG_PREV_CREDIT               16454    5.350703
DAYS_ENTRY_PAYMENT            16454    5.350703
NAME_TYPE_SUITE 

## Numerical Moderate missing (5–30%)
---

In [13]:
# Moderate missing (5-30%)

moderate_missing_numerical = missing_numerical[(missing_numerical >= 5) & (missing_numerical <= 30)]
print("Numerical Features with Moderate Missing Values (5% <= Missing <= 30%):\n", moderate_missing_numerical)

Numerical Features with Moderate Missing Values (5% <= Missing <= 30%):
 NUM_BUREAU_LOANS      14.314935
SUM_CREDIT            14.314935
AVG_CREDIT            14.315260
FIRST_CREDIT_DAYS     14.314935
LAST_CREDIT_DAYS      14.314935
ACTIVE_LOANS          14.314935
OVERDUE_MONTHS        14.314935
NUM_PREV_LOANS         5.350703
SUM_PREV_CREDIT        5.350703
AVG_PREV_CREDIT        5.350703
RECENT_LOAN_DAYS       5.350703
MONTHS_BALANCE_min     6.680737
MONTHS_BALANCE_max     6.680737
AMT_PAYMENT            5.890196
DAYS_ENTRY_PAYMENT     5.350703
dtype: float64


In [14]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define function to fill missing values using LightGBM
def impute_with_lgbm(data, target_col, n_estimators=100, learning_rate=0.05):
    try:
        # Separate rows with missing and non-missing values
        missing_mask = data[target_col].isnull()
        known_data = data[~missing_mask]
        unknown_data = data[missing_mask]

        if known_data.shape[0] == 0:  # If no known data, skip
            print(f"Skipping imputation for {target_col} - No known data.")
            return data

        # Identify categorical and numerical features
        categorical_features = data.select_dtypes(include=['object', 'category']).columns.tolist()
        categorical_features = [col for col in categorical_features if col != target_col and col in data.columns]
        numerical_features = data.select_dtypes(include=['number']).columns.tolist()
        numerical_features = [col for col in numerical_features if col != target_col and col in data.columns]

        # Use all other numerical and encoded categorical columns as features
        feature_cols = numerical_features + categorical_features

        # Create a copy of the data to avoid modifying the original DataFrame
        data_encoded = data.copy()

        # Apply label encoding to categorical features
        encoders = {}
        for feature in categorical_features:
            le = LabelEncoder()
            data_encoded[feature] = le.fit_transform(data_encoded[feature])
            encoders[feature] = le  # Store the encoders for later decoding

        # Train-Test Split using encoded data
        X_train, X_valid, y_train, y_valid = train_test_split(
            data_encoded.loc[~missing_mask, feature_cols],
            data_encoded.loc[~missing_mask, target_col],
            test_size=0.2,
            random_state=42
        )

        # Train LightGBM model
        model = lgb.LGBMRegressor(n_estimators=n_estimators, learning_rate=learning_rate)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_valid, y_valid)],
            callbacks=[early_stopping(stopping_rounds=10), log_evaluation(period=10)],
        )

        # Predict missing values using encoded data
        imputed_values = model.predict(data_encoded.loc[missing_mask, feature_cols])
        data.loc[missing_mask, target_col] = imputed_values  # Impute directly into original data

        # Check for missing values after imputation
        num_missing_after = data[target_col].isnull().sum()
        print(f"Number of missing values in {target_col} after imputation: {num_missing_after}")

        return data

    except Exception as e:
        print(f"Error imputing {target_col}: {e}")
        return data  # Return original data to avoid further issues

# Add sleep after training
time.sleep(5)  # Sleep for 5 seconds to allow cache flushing

In [15]:
features_to_impute = [
    'NUM_BUREAU_LOANS',
    'SUM_CREDIT',
    'AVG_CREDIT',
    'FIRST_CREDIT_DAYS',
    'LAST_CREDIT_DAYS',
    'ACTIVE_LOANS',
    'OVERDUE_MONTHS',
    'NUM_PREV_LOANS',
    'SUM_PREV_CREDIT',
    'AVG_PREV_CREDIT',
    'RECENT_LOAN_DAYS',
    'MONTHS_BALANCE_min',
    'MONTHS_BALANCE_max',
    'AMT_PAYMENT',
    'DAYS_ENTRY_PAYMENT'
]


for col in features_to_impute:
    # Create a new column name for the imputed values
    imputed_col_name = col + '_imputed'

    # Impute missing values and store in the new column
    clean_application_train[imputed_col_name] = impute_with_lgbm(clean_application_train, col)[col]

# Add sleep after training
time.sleep(5)  # Sleep for 5 seconds to allow cache flushing

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014903 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5377
[LightGBM] [Info] Number of data points in the train set: 210792, number of used features: 46
[LightGBM] [Info] Start training from score 5.558010
Training until validation scores don't improve for 10 rounds
[10]	valid_0's l2: 9.62196
[20]	valid_0's l2: 5.07528
[30]	valid_0's l2: 2.61926
[40]	valid_0's l2: 1.46671
[50]	valid_0's l2: 0.923218
[60]	valid_0's l2: 0.648818
[70]	valid_0's l2: 0.489656
[80]	valid_0's l2: 0.389986
[90]	valid_0's l2: 0.323502
[100]	valid_0's l2: 0.279324
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.279324
Number of missing values in NUM_BUREAU_LOANS after imputation: 0
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012322 seconds.
You can s

## Numerical High missing (>30%)
---

In [16]:
# High missing (>30%)

high_missing_numerical = missing_numerical[missing_numerical > 30]

# Get only the feature names (index)
high_missing_features = high_missing_numerical.index.tolist()

print("Numerical Features with High Missing Values (> 30%):\n", high_missing_features)

Numerical Features with High Missing Values (> 30%):
 ['OWN_CAR_AGE', 'FIRST_MONTH_BALANCE', 'LAST_MONTH_BALANCE']


In [17]:
# Drop low-importance features, but only if they exist
drop_cols = ['OWN_CAR_AGE', 'FIRST_MONTH_BALANCE', 'LAST_MONTH_BALANCE']

# Check if columns exist in clean_application_train before dropping
cols_to_drop = [col for col in drop_cols if col in clean_application_train.columns]

# Drop columns from clean_application_train (if they exist)
if cols_to_drop:
    clean_application_train.drop(columns=cols_to_drop, inplace=True)
else:
    print("Columns to drop not found in clean_application_train DataFrame.")

In [18]:
clean_application_train.shape

(307511, 60)

In [19]:
clean_application_train.isnull().sum()

SK_ID_CURR                        0
TARGET                            0
NAME_CONTRACT_TYPE                0
FLAG_OWN_CAR                      0
FLAG_OWN_REALTY                   0
CNT_CHILDREN                      0
AMT_INCOME_TOTAL                  0
AMT_CREDIT                        0
AMT_ANNUITY                       0
AMT_GOODS_PRICE                   0
NAME_TYPE_SUITE                1292
NAME_INCOME_TYPE                  0
NAME_EDUCATION_TYPE               0
NAME_FAMILY_STATUS                0
NAME_HOUSING_TYPE                 0
DAYS_BIRTH                        0
DAYS_EMPLOYED                     0
DAYS_REGISTRATION                 0
DAYS_ID_PUBLISH                   0
FLAG_MOBIL                        0
FLAG_EMP_PHONE                    0
FLAG_WORK_PHONE                   0
FLAG_CONT_MOBILE                  0
FLAG_PHONE                        0
FLAG_EMAIL                        0
OCCUPATION_TYPE               96391
CNT_FAM_MEMBERS                   0
WEEKDAY_APPR_PROCESS_START  

## Categorical Low missing (<5%)
---

In [21]:
# Hitung persentase missing values untuk kolom bertipe kategorikal
missing_categorical = clean_application_train.select_dtypes(include=['object', 'category']).isnull().mean() * 100

# Low missing (<5%)

low_missing_categorical = missing_categorical[(missing_categorical < 5) & (missing_categorical > 0)]
print("Numerical Features with Low Missing Values (0% < Missing < 5%):\n", low_missing_categorical)

Numerical Features with Low Missing Values (0% < Missing < 5%):
 NAME_TYPE_SUITE    0.420148
dtype: float64


In [22]:
# fill categorical features with low missing values (less than 5%) using the mode

for feature in low_missing_categorical.index:
    # Check if the column already has imputed values (e.g., '_imputed' suffix)
    imputed_col_name = feature + '_imputed'
    if imputed_col_name not in clean_application_train.columns:
        # Impute missing values and store in a new column with '_imputed' suffix
        clean_application_train[imputed_col_name] = clean_application_train[feature].fillna(clean_application_train[feature].mode()[0])
    else:
        # If imputed column already exists, use it
        print(f"Column '{feature}' already has imputed values in '{imputed_col_name}'. Skipping imputation.")

    # Replace original column with imputed column
    clean_application_train[feature] = clean_application_train[imputed_col_name]
    clean_application_train.drop(columns=[imputed_col_name], inplace=True) # Remove the temporary imputed column

In [23]:
check_missing_values(clean_application_train)

Missing Values Summary:
                 Missing Values  Percentage
OCCUPATION_TYPE           96391   31.345545


## Moderate missing (5–30%)
---

In [24]:
# Moderate missing (5-30%)

moderate_missing_categorical = missing_categorical[(missing_numerical >= 5) & (missing_categorical <= 30)]
print("Numerical Features with Moderate Missing Values (5% <= Missing <= 30%):\n", moderate_missing_categorical)

Numerical Features with Moderate Missing Values (5% <= Missing <= 30%):
 Series([], dtype: float64)


## High missing (>30%)
---

In [25]:
# High missing (>30%)

high_missing_categorical = missing_categorical[missing_categorical > 30]
print("Categorical Features with High Missing Values (> 30%):\n", high_missing_categorical)

Categorical Features with High Missing Values (> 30%):
 OCCUPATION_TYPE    31.345545
dtype: float64


In [26]:
clean_application_train['OCCUPATION_TYPE'].fillna('Other', inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7816\2434723297.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clean_application_train['OCCUPATION_TYPE'].fillna('Other', inplace=True)


In [27]:
check_missing_values(clean_application_train)

No missing values found in the DataFrame.


In [28]:
clean_application_train.to_csv('clean_application_train.csv', index=False)

## <b>3.  Detecting and Handling Outliers</font></b>
---
#### Use IQR to Detect Outliers




In [30]:
outlier_application_train = clean_application_train

In [31]:
def detect_outliers_iqr(df, columns):
    outlier_info = {}  # Dictionary to store features and outlier numbers
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        num_outliers = len(outliers)  # Get the number of outliers

        if num_outliers > 0:  # Check if there are any outliers for this column
            outlier_info[col] = num_outliers  # Store feature and outlier number

    return outlier_info  # Return the dictionary

In [32]:
# Select numerical columns from outlier_application_train
num_cols = outlier_application_train.select_dtypes(include=[np.number]).columns

# Detect outliers using outlier_application_train and get features with outlier numbers
outliers_iqr = detect_outliers_iqr(outlier_application_train, num_cols)
outliers_iqr

{'TARGET': 24825,
 'CNT_CHILDREN': 4272,
 'AMT_INCOME_TOTAL': 14035,
 'AMT_CREDIT': 6562,
 'AMT_ANNUITY': 7504,
 'AMT_GOODS_PRICE': 14728,
 'DAYS_EMPLOYED': 72217,
 'DAYS_REGISTRATION': 659,
 'FLAG_MOBIL': 1,
 'FLAG_EMP_PHONE': 55386,
 'FLAG_WORK_PHONE': 61308,
 'FLAG_CONT_MOBILE': 574,
 'FLAG_EMAIL': 17442,
 'CNT_FAM_MEMBERS': 4007,
 'DAYS_LAST_PHONE_CHANGE': 435,
 'NUM_BUREAU_LOANS': 9094,
 'SUM_CREDIT': 26131,
 'AVG_CREDIT': 31486,
 'LAST_CREDIT_DAYS': 29931,
 'ACTIVE_LOANS': 6454,
 'OVERDUE_MONTHS': 65732,
 'NUM_PREV_LOANS': 15426,
 'SUM_PREV_CREDIT': 26567,
 'AVG_PREV_CREDIT': 19924,
 'MONTHS_BALANCE_max': 27934,
 'AMT_PAYMENT': 30468,
 'NUM_BUREAU_LOANS_imputed': 9094,
 'SUM_CREDIT_imputed': 26131,
 'AVG_CREDIT_imputed': 31486,
 'LAST_CREDIT_DAYS_imputed': 29931,
 'ACTIVE_LOANS_imputed': 6454,
 'OVERDUE_MONTHS_imputed': 65732,
 'NUM_PREV_LOANS_imputed': 15426,
 'SUM_PREV_CREDIT_imputed': 26567,
 'AVG_PREV_CREDIT_imputed': 19924,
 'MONTHS_BALANCE_max_imputed': 27934,
 'AMT_PAYMENT

## Handle Outlier
---
#### Financial Features (Key Loan & Income Variables)

In [33]:
financial_features = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE']

for col in financial_features:
    lower_limit = outlier_application_train[col].quantile(0.01)
    upper_limit = outlier_application_train[col].quantile(0.99)

    # Replace values directly in outlier_application_train
    outlier_application_train[col] = np.clip(outlier_application_train[col], lower_limit, upper_limit)

# Check for changes
for col in financial_features:
    before_range = outlier_application_train[col].describe()[['min', 'max']]  # No need for a copy anymore
    print(f"Column: {col}")
    print(f"Range after clipping: {before_range}")  # Print the range after clipping
    print("-" * 20)

Column: AMT_INCOME_TOTAL
Range after clipping: min     9321750.0
max    97878375.0
Name: AMT_INCOME_TOTAL, dtype: float64
--------------------
Column: AMT_CREDIT
Range after clipping: min     15828331.5
max    384056100.0
Name: AMT_CREDIT, dtype: float64
--------------------
Column: AMT_ANNUITY
Range after clipping: min    1.280808e+06
max    1.450185e+07
Name: AMT_ANNUITY, dtype: float64
--------------------
Column: AMT_GOODS_PRICE
Range after clipping: min     13982625.0
max    372870000.0
Name: AMT_GOODS_PRICE, dtype: float64
--------------------


In [34]:
outliers_iqr = detect_outliers_iqr(outlier_application_train, num_cols)
outliers_iqr

{'TARGET': 24825,
 'CNT_CHILDREN': 4272,
 'AMT_INCOME_TOTAL': 14035,
 'AMT_CREDIT': 6562,
 'AMT_ANNUITY': 7504,
 'AMT_GOODS_PRICE': 14728,
 'DAYS_EMPLOYED': 72217,
 'DAYS_REGISTRATION': 659,
 'FLAG_MOBIL': 1,
 'FLAG_EMP_PHONE': 55386,
 'FLAG_WORK_PHONE': 61308,
 'FLAG_CONT_MOBILE': 574,
 'FLAG_EMAIL': 17442,
 'CNT_FAM_MEMBERS': 4007,
 'DAYS_LAST_PHONE_CHANGE': 435,
 'NUM_BUREAU_LOANS': 9094,
 'SUM_CREDIT': 26131,
 'AVG_CREDIT': 31486,
 'LAST_CREDIT_DAYS': 29931,
 'ACTIVE_LOANS': 6454,
 'OVERDUE_MONTHS': 65732,
 'NUM_PREV_LOANS': 15426,
 'SUM_PREV_CREDIT': 26567,
 'AVG_PREV_CREDIT': 19924,
 'MONTHS_BALANCE_max': 27934,
 'AMT_PAYMENT': 30468,
 'NUM_BUREAU_LOANS_imputed': 9094,
 'SUM_CREDIT_imputed': 26131,
 'AVG_CREDIT_imputed': 31486,
 'LAST_CREDIT_DAYS_imputed': 29931,
 'ACTIVE_LOANS_imputed': 6454,
 'OVERDUE_MONTHS_imputed': 65732,
 'NUM_PREV_LOANS_imputed': 15426,
 'SUM_PREV_CREDIT_imputed': 26567,
 'AVG_PREV_CREDIT_imputed': 19924,
 'MONTHS_BALANCE_max_imputed': 27934,
 'AMT_PAYMENT

In [35]:
# Create a copy to avoid modifying the original (if not already done)
outlier_application_train_cleaned = outlier_application_train.copy()

# Log transform DAYS_EMPLOYED and store in a new column
outlier_application_train_cleaned['DAYS_EMPLOYED_log'] = np.log1p(outlier_application_train_cleaned['DAYS_EMPLOYED'].abs())

# Replace the original column with the log-transformed values
outlier_application_train_cleaned['DAYS_EMPLOYED'] = outlier_application_train_cleaned['DAYS_EMPLOYED_log']
outlier_application_train_cleaned.drop(columns=['DAYS_EMPLOYED_log'], inplace=True)

# Assign the cleaned DataFrame back to the original
outlier_application_train = outlier_application_train_cleaned

# Check descriptive statistics before and after transformation
after_stats = outlier_application_train_cleaned['DAYS_EMPLOYED'].describe()  # Can still use outlier_application_train_cleaned

print("DAYS_EMPLOYED Statistics:")
print("After Transformation:", after_stats)

DAYS_EMPLOYED Statistics:
After Transformation: count    307511.000000
mean          8.294337
std           2.321902
min           0.000000
25%           6.839476
50%           7.705262
75%           8.649624
max          12.808321
Name: DAYS_EMPLOYED, dtype: float64


In [36]:
credit_features = ['AMT_REQ_CREDIT_BUREAU_YEAR', 'NUM_BUREAU_LOANS']

# Create a copy to avoid modifying the original
outlier_application_train_cleaned = outlier_application_train.copy()

for col in credit_features:
    # Check if the column exists before processing
    if col in outlier_application_train_cleaned.columns:
        # Calculate quantiles for clipping
        lower_limit = outlier_application_train_cleaned[col].quantile(0.01)
        upper_limit = outlier_application_train_cleaned[col].quantile(0.99)

        # Clip values directly in the outlier_application_train DataFrame
        outlier_application_train[col] = np.clip(outlier_application_train[col], lower_limit, upper_limit)
    else:
        print(f"Column '{col}' not found in DataFrame. Skipping outlier handling.")

# --- Code to check the results ---

for col in credit_features:
    if col in outlier_application_train.columns:
        before_range = outlier_application_train_cleaned[col].describe()[['min', 'max']]  # Original range
        after_range = outlier_application_train[col].describe()[['min', 'max']]  # Clipped range
        print(f"Column: {col}")
        print(f"Range before clipping: {before_range}")
        print(f"Range after clipping: {after_range}")
        print("-" * 20)
    else:
        print(f"Column '{col}' not found for range comparison.")

Column 'AMT_REQ_CREDIT_BUREAU_YEAR' not found in DataFrame. Skipping outlier handling.
Column 'AMT_REQ_CREDIT_BUREAU_YEAR' not found for range comparison.
Column: NUM_BUREAU_LOANS
Range before clipping: min      1.0
max    116.0
Name: NUM_BUREAU_LOANS, dtype: float64
Range after clipping: min     1.0
max    20.0
Name: NUM_BUREAU_LOANS, dtype: float64
--------------------


In [37]:
loan_features = ['OVERDUE_MONTHS', 'LAST_CREDIT_DAYS', 'AMT_PAYMENT']

# Create a copy to avoid modifying the original
outlier_application_train_cleaned = outlier_application_train.copy()

for col in loan_features:
    # Check if the column exists before processing
    if col in outlier_application_train_cleaned.columns:
        # Calculate quantiles for clipping
        lower_limit = outlier_application_train_cleaned[col].quantile(0.01)
        upper_limit = outlier_application_train_cleaned[col].quantile(0.99)

        # Clip values directly in the outlier_application_train DataFrame
        outlier_application_train[col] = np.clip(outlier_application_train[col], lower_limit, upper_limit)
    else:
        print(f"Column '{col}' not found in DataFrame. Skipping outlier handling.")

# --- Code to check the results ---

for col in loan_features:
    if col in outlier_application_train.columns:
        before_range = outlier_application_train_cleaned[col].describe()[['min', 'max']]  # Original range
        after_range = outlier_application_train[col].describe()[['min', 'max']]  # Clipped range
        print(f"Column: {col}")
        print(f"Range before clipping: {before_range}")
        print(f"Range after clipping: {after_range}")
        print("-" * 20)
    else:
        print(f"Column '{col}' not found for range comparison.")

Column: OVERDUE_MONTHS
Range before clipping: min       0.0
max    2252.0
Name: OVERDUE_MONTHS, dtype: float64
Range after clipping: min      0.0
max    414.0
Name: OVERDUE_MONTHS, dtype: float64
--------------------
Column: LAST_CREDIT_DAYS
Range before clipping: min   -2922.0
max       0.0
Name: LAST_CREDIT_DAYS, dtype: float64
Range after clipping: min   -2564.0
max     -16.0
Name: LAST_CREDIT_DAYS, dtype: float64
--------------------
Column: AMT_PAYMENT
Range before clipping: min    3.293685e+03
max    5.188257e+08
Name: AMT_PAYMENT, dtype: float64
Range after clipping: min    5.430207e+05
max    3.680604e+07
Name: AMT_PAYMENT, dtype: float64
--------------------


In [41]:
# outlier_application_train.to_csv('outlier_application_train.csv', index=False)

In [42]:
outlier_application_train.shape # SHAPE NYA JADI (307511, 45)

(307511, 60)

In [43]:
clean_application_train.isnull().sum()

SK_ID_CURR                    0
TARGET                        0
NAME_CONTRACT_TYPE            0
FLAG_OWN_CAR                  0
FLAG_OWN_REALTY               0
CNT_CHILDREN                  0
AMT_INCOME_TOTAL              0
AMT_CREDIT                    0
AMT_ANNUITY                   0
AMT_GOODS_PRICE               0
NAME_TYPE_SUITE               0
NAME_INCOME_TYPE              0
NAME_EDUCATION_TYPE           0
NAME_FAMILY_STATUS            0
NAME_HOUSING_TYPE             0
DAYS_BIRTH                    0
DAYS_EMPLOYED                 0
DAYS_REGISTRATION             0
DAYS_ID_PUBLISH               0
FLAG_MOBIL                    0
FLAG_EMP_PHONE                0
FLAG_WORK_PHONE               0
FLAG_CONT_MOBILE              0
FLAG_PHONE                    0
FLAG_EMAIL                    0
OCCUPATION_TYPE               0
CNT_FAM_MEMBERS               0
WEEKDAY_APPR_PROCESS_START    0
ORGANIZATION_TYPE             0
DAYS_LAST_PHONE_CHANGE        0
NUM_BUREAU_LOANS              0
SUM_CRED